<a href="https://colab.research.google.com/github/sainvo/DeepLearning_NER/blob/master/DL_NER_update2_unfinished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning NER task

Tatjana Cucic and Sanna Volanen

https://spacy.io/api/annotation

# Milestones

## 1.1 Predicting word labels independently

* The first part is to train a classifier which assigns a label for each given input word independently. 
* Evaluate the results on token level and entity level. 
* Report your results with different network hyperparameters. 
* Also discuss whether the token level accuracy is a reasonable metric.









In [3]:
# Training data: Used for training the model
!wget https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/train.tsv

# Development/ validation data: Used for testing different model parameters, for example level of regularization needed
!wget https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/dev.tsv

# Test data: Never touched during training / model development, used for evaluating the final model
!wget https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/test.tsv

import sys 
import csv

csv.field_size_limit(sys.maxsize)

--2020-05-08 10:29:52--  https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17252156 (16M) [text/plain]
Saving to: ‘train.tsv.1’

train.tsv.1         100%[===================>]  16.45M  31.8MB/s    in 0.5s    

2020-05-08 10:29:53 (31.8 MB/s) - ‘train.tsv.1’ saved [17252156/17252156]

--2020-05-08 10:29:54--  https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2419425 (2.3M) [text/plain]
Saving to: ‘de

131072

In [4]:
from collections import namedtuple
OneWord=namedtuple("OneWord",["word","entity_label"])

def read_ontonotes(tsv_file):
  #"""Yield complete sentences"""
    current_sentence=[] # list of (word,label) tuples
    with open(tsv_file) as f:
        tsvreader = csv.reader(f, delimiter= '\t')
        for line in tsvreader:
            #print(line)
            if not line: #sentence break
                if current_sentence: #if we gathered a sentence, we should yield it, because a new starts
                    yield current_sentence #much like return, but continues past this line once the element has been consumed
                    current_sentence=[] #...and start a new one
                continue
            #if we made it here, we are on a normal line
            columns=[line[0], line[1]] #an actual word line
            assert len(columns)==2 #we should have four columns, looking at the data
            current_sentence.append(OneWord(*columns)) #shorthand for looping over columns
        else: #for ... else -> the else part is executed once, when "for" runs out of elements
            if current_sentence: #yield also the last one!
                yield current_sentence

#read the data in as sentences
sentences_train=list(read_ontonotes("train.tsv"))
sentences_dev=list(read_ontonotes("dev.tsv"))
sentences_test = list(read_ontonotes("test.tsv"))

print(type(sentences_test))

print("First three sentences")
for sent in sentences_train[:3]:
    print(sent)
print(len(sentences_train))
print('---------------------------------------------')
print("First three sentences")
for sent in sentences_dev[:3]:
    print(sent)
print(len(sentences_dev))
print('---------------------------------------------')
print("First three sentences")
for sent in sentences_test[:3]:
    print(sent)
print(len(sentences_test))

<class 'list'>
First three sentences
[OneWord(word='Big', entity_label='O'), OneWord(word='Managers', entity_label='O'), OneWord(word='on', entity_label='O'), OneWord(word='Campus', entity_label='O')]
[OneWord(word='In', entity_label='O'), OneWord(word='recent', entity_label='B-DATE'), OneWord(word='years', entity_label='I-DATE'), OneWord(word=',', entity_label='O'), OneWord(word='advanced', entity_label='O'), OneWord(word='education', entity_label='O'), OneWord(word='for', entity_label='O'), OneWord(word='professionals', entity_label='O'), OneWord(word='has', entity_label='O'), OneWord(word='become', entity_label='O'), OneWord(word='a', entity_label='O'), OneWord(word='hot', entity_label='O'), OneWord(word='topic', entity_label='O'), OneWord(word='in', entity_label='O'), OneWord(word='the', entity_label='O'), OneWord(word='business', entity_label='O'), OneWord(word='community', entity_label='O'), OneWord(word='.', entity_label='O')]
[OneWord(word='With', entity_label='O'), OneWord(wor

In [0]:
# shape into dicts per sentence

def reshape_sent2dicts(f):
    #data_dict = []
    for line in f:
        sent_text= []
        sent_tags = []
        for OneWord in line:
            #print(OneWord)
            sent_text.append(OneWord.word)
            sent_tags.append(OneWord.entity_label)
        sent_dict = {'text':sent_text,'tags':sent_tags }
        #print(sent_dict)
        #data_dict_train.append(sent_dict)
    return sent_dict

train_data = list(reshape_sent2dicts(sentences_train[:30000]))

dev_data = reshape_sent2dicts(sentences_dev)

In [8]:
import random
import numpy

random.seed(124)
random.shuffle(train_data)
print(type(train_data))
print(type(train_data[0]))

train_texts=[[j[] for j in i] for i in train_data]
train_labels=[[j["tags"] for j in i] for i in train_data]

print('Text: ', train_texts[0])
print('Label: ', train_labels[0])

<class 'list'>
<class 'str'>


TypeError: ignored

In [0]:
# Load pretrained embeddings
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


In [0]:
# Give -n argument so that a possible existing file isn't overwritten 
!unzip -n wiki-news-300d-1M.vec.zip

In [0]:
from gensim.models import KeyedVectors

vector_model = KeyedVectors.load_word2vec_format("wiki-news-300d-1M.vec", binary=False, limit=50000)


# sort based on the index to make sure they are in the correct order
words = [k for k, v in sorted(vector_model.vocab.items(), key=lambda x: x[1].index)]
print("Words from embedding model:", len(words))
print("First 50 words:", words[:50])

# Normalize the vectors to unit length
print("Before normalization:", vector_model.get_vector("in")[:10])
vector_model.init_sims(replace=True)
print("After normalization:", vector_model.get_vector("in")[:10])

In [0]:
# Build vocabulary mappings

# Zero is used for padding in Keras, prevent using it for a normal word.
# Also reserve an index for out-of-vocabulary items.
vocabulary={
    "<PAD>": 0,
    "<OOV>": 1
}

for word in words: # These are words from the word2vec model
    vocabulary.setdefault(word, len(vocabulary))

print("Words in vocabulary:",len(vocabulary))
inv_vocabulary = { value: key for key, value in vocabulary.items() } # invert the dictionary


# Embedding matrix
def load_pretrained_embeddings(vocab, embedding_model):
    """ vocab: vocabulary from our data vectorizer, embedding_model: model loaded with gensim """
    pretrained_embeddings = numpy.random.uniform(low=-0.05, high=0.05, size=(len(vocab)-1,embedding_model.vectors.shape[1]))
    pretrained_embeddings = numpy.vstack((numpy.zeros(shape=(1,embedding_model.vectors.shape[1])), pretrained_embeddings))
    found=0
    for word,idx in vocab.items():
        if word in embedding_model.vocab:
            pretrained_embeddings[idx]=embedding_model.get_vector(word)
            found+=1
            
    print("Found pretrained vectors for {found} words.".format(found=found))
    return pretrained_embeddings

pretrained=load_pretrained_embeddings(vocabulary, vector_model)

Preprocessing

In [0]:
#Labels
from pprint import pprint


# Label mappings
# 1) gather a set of unique labels
label_set = set()
for sentence_labels in train_labels: #loops over sentences 
    for label in sentence_labels: #loops over labels in one sentence
        label_set.add(label)

# 2) index these
label_map = {}
for index, label in enumerate(label_set):
    label_map[label]=index
    
print(label_map)

In [0]:
# vectorize the labels
def label_vectorizer(labels,label_map):
    vectorized_labels = []
    for label in labels:
        vectorized_example_label = []
        for token in label:
            vectorized_example_label.append(label_map[token])
        vectorized_labels.append(vectorized_example_label)
    vectorized_labels = numpy.array(vectorized_labels)
    return vectorized_labels
        

vectorized_labels = label_vectorizer(labels,label_map)
validation_vectorized_labels = label_vectorizer(validation_labels,label_map)

print(vectorized_labels[0])

## 1.2 Expand context

Modify your network in such way that it is able to utilize the surrounding context of the word. This can be done for instance with a convolutional or recurrent layer. Analyze different neural network architectures and hyperparameters. How does utilizing the surrounding context influence the predictions?


## 2.1 Use deep contextual representations

Use deep contextual representations. Fine-tune the embeddings with different hyperparameters. Try different models (e.g. cased and uncased, multilingual BERT). Report your results.


## 2.2 Error analysis

Select one model from each of the previous milestones (three models in total). Look at the entities these models predict. Analyze the errors made. Are there any patterns? How do the errors one model makes differ from those made by another?

## 3.1 Predictions on unannotated text

Use the three models selected in milestone 2.2 to do predictions on the sampled wikipedia text.

## 3.2 Statistically analyze the results

Statistically analyze (i.e. count the number of instances) and compare the predictions. You can, for example, analyze if some models tend to predict more entities starting with a capital letter, or if some models predict more entities for some specific classes than others.